# 03. 토치텍스트 튜토리얼(Torchtext tutorial) - 한국어
---

In [ ]:
# 한국어로 진행해보자

In [13]:
# 훈련 데이터와 테스트 데이터 다운로드
import urllib.request
import pandas as pd

In [3]:
datapath = 'D:/chch/startwhithtorch/datafolder/'
urllib.request.urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', filename = datapath + "rating_train.txt")
urllib.request.urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', filename = datapath + "rating_test.txt")

('D:/chch/startwhithtorch/datafolder/rating_test.txt',
 <http.client.HTTPMessage at 0x176f5663880>)

In [4]:
# 다운받은 txt 데이터를 df 형태로 매핑
train_df = pd.read_table(datapath + "rating_train.txt")
test_df = pd.read_table(datapath + "rating_test.txt")

In [14]:
# 훈련 데이터 확인
print(train_df.head())
print(test_df.head())

         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
        id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0


In [15]:
# 샘플 개수 확인
print('훈련 데이터 샘플의 개수: {}'.format(len(train_df)))
print('테스트 데이터 샘플의 개수: {}'.format(len(test_df)))

훈련 데이터 샘플의 개수: 150000
테스트 데이터 샘플의 개수: 50000


### 3. 필드 정의하기(torchtext.data)
---

In [20]:
from torchtext.legacy import data
from eunjeon import Mecab

# 토크나이저(토큰화 도구)는 Mecab을 이용
tokenizer = Mecab()

In [22]:
# 네이버 영화 리뷰 데이터는 3개의 열로 구성되어 있으니 3개의 필드를 정의

# id 필드 정의
ID = data.Field(sequential=False,  # 시퀀스 아님
                use_vocab = False) # 단어 집합 만들지 않음

# text data 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab = True,
                  tokenize = tokenizer.morphs,
                  lower = True,
                  batch_first = True,
                  fix_length = 20)

# label 필드 정의
LABEL = data.Field(sequential=False,
                   use_vocab = False,
                   is_target=True)   

### 4. 데이터셋 만들기
---

In [23]:
from torchtext.legacy.data import TabularDataset

In [26]:
# 데이터를 불러와서 데이터셋의 형식으로 바꾸면서 토큰화까지 진행하자
train_data, test_data = TabularDataset.splits(path=datapath, train = 'rating_train.txt', test = 'rating_test.txt',
                                              format = 'tsv', 
                                              fields = [('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True)

In [27]:
# 데이터셋으로 변환된 샘플의 개수 확인
print('훈련 데이터 샘플의 개수: {}'.format(len(train_data)))
print('테스트 데이터 샘플의 개수: {}'.format(len(test_data)))

# 앞서 확인한 개수와 동일할 거임

훈련 데이터 샘플의 개수: 150000
테스트 데이터 샘플의 개수: 50000


In [28]:
# 학습데이터의 첫번째 샘플을 출력해보자
print(vars(train_data[0]))

{'id': '9976970', 'text': ['아', '더빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], 'label': '0'}


### 5. 단어 집합(Vocabulary) 만들기
---

In [29]:
# 정의한 필드에 .build_vocab() 도구를 사용해 단어집합을 생성하자
TEXT.build_vocab(train_data, min_freq = 10, max_size = 10000)

In [30]:
# 생성된 단어 집합의 크기 확인
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


In [ ]:
# .stoi 로 단어 집합 내 단어를 확인
print(TEXT.vocab.stoi)

### 6. 토치텍스트의 데이터로더 만들기
---

In [39]:
# Iterator를 사용해 미니배치만큼 데이터를 둘러오는 데이터로더 만들기

from torchtext.legacy.data import Iterator

In [40]:
# 임의 배치 지정
batch_size = 5

train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

In [41]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 30000
테스트 데이터의 미니 배치 수 : 10000


In [43]:
# 첫번째 미니 배치를 받아 batch에 저장하고 첫번쨰 미니 배치의 test 필드를 호출하자
batch = next(iter(train_loader))
print(batch.text)

tensor([[1702,    8,   18, 3515,    8,   18,  576,    8, 1733,  108,   36,   15,
         1733,  114,   45,   80,   36,   15,   58,  392],
        [ 256,    0,  788, 2906,   23, 1119,  992,  116,    6,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [ 976,   10, 2067, 1473,   60,  251,   91,   96,  118,  304,    6,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   0,    0,  316,  118,  424,    0,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [6579, 1335,    4, 1375,   99,   30,   49,   31,  142,  495,   80,  810,
           14,  884,   36, 1843,    1,    1,    1,    1]])


In [ ]:
# unk 는 0
# padding 은 1 로 설정된 것을 확인